In [1]:
from pyelpa import DistributedMatrix
import numpy as np
import h5py
import sys


sys.path.append(".")
dirname = "./bgw_files/k-4x4x4/"

In [2]:
def print_2d_matrix(matrix, decimals=3):
    for row in matrix:
        print(" ".join(f"{val:9.{decimals}f}" for val in row))

In [3]:
# epsinp and sigmain data.
from qtm.interfaces.bgw.epsinp import Epsinp
epsinp = Epsinp.from_epsilon_inp(filename=dirname+'epsilon.inp')

from qtm.interfaces.bgw.sigmainp import Sigmainp
sigmainp = Sigmainp.from_sigma_inp(filename=dirname+'sigma.inp')

In [4]:
from qtm.interfaces.bgw import inp
from qtm.interfaces.bgw.wfn2py import wfn2py

# WFN data
wfndata = wfn2py(dirname+'WFN.h5')
wfnqdata = wfn2py(dirname+'WFNq.h5')

# RHO data
rho = inp.read_rho(dirname+"RHO")

# Vxc data
vxc = inp.read_vxc(dirname+"vxc.dat")

In [5]:
from qtm.gw.core import QPoints
from qtm.gw.epsilon import Epsilon

epsilon = Epsilon.from_data(wfndata=wfndata, wfnqdata=wfnqdata, epsinp=epsinp)

Vcoul calculation for qpts: 100%|██████████| 64/64 [00:00<00:00, 18089.86it/s]


In [6]:
from tqdm import trange
from qtm.gw.core import reorder_2d_matrix_sorted_gvecs, sort_cryst_like_BGW


def calculate_epsilon(numq=None, writing=False):
    epsmats = []
    if numq is None:
        numq = epsilon.qpts.numq

    for i_q in trange(0, numq, desc="Epsilon> q-pt index"):
        # Create map between BGW's sorting order and QTm's sorting order
        gkspc = epsilon.l_gq[i_q]
        
        if i_q == epsilon.qpts.index_q0:
            key = gkspc.g_norm2
        else:
            key = gkspc.gk_norm2

        indices_gspace_sorted = sort_cryst_like_BGW(
            cryst=gkspc.g_cryst, key_array=key
        )

        # Calculate matrix elements
        M = next(epsilon.matrix_elements(i_q=i_q))

        # Calculate polarizability matrix (faster, but not memory-efficient)
        chimat = epsilon.polarizability(M)

        # Calculate polarizability matrix (memory-efficient)
        # chimat = epsilon.polarizability_active(i_q)

        # Calculate epsilon inverse matrix
        epsinv = epsilon.epsilon_inverse(i_q=i_q, polarizability_matrix=chimat, store=True)


        epsinv = reorder_2d_matrix_sorted_gvecs(epsinv, indices_gspace_sorted)
        epsilon.l_epsinv[i_q] = epsinv
        
        # Compare the results with BGW's results
        if i_q == epsilon.qpts.index_q0:
            epsref = epsilon.read_epsmat(dirname + "eps0mat.h5")[0][0, 0]
            if writing:
                epsilon.write_epsmat(
                    filename="test/epsilon/eps0mat_qtm.h5", epsinvmats=[epsinv]
                )
        else:
            epsref = np.array(epsilon.read_epsmat(dirname + "epsmat.h5")[i_q - 1][0, 0])
            epsmats.append(epsinv)

        # Calculate stddev between reference and calculated epsinv matrices
        std_eps = np.std(epsref - epsinv) / np.sqrt(np.prod(list(epsinv.shape)))

        epstol = 1e-16
        if np.abs(std_eps) > epstol:
            print(f"Standard deviation exceeded {epstol} tolerance: {std_eps}, for i_q:{i_q}")

    if writing:
        epsilon.write_epsmat(filename="test/epsilon/epsmat_qtm.h5", epsinvmats=epsmats)


epsinp.no_min_fftgrid = True
epsilon = Epsilon.from_data(wfndata=wfndata, wfnqdata=wfnqdata, epsinp=epsinp)
calculate_epsilon()

Epsilon> q-pt index: 100%|██████████| 64/64 [00:18<00:00,  3.41it/s]


In [7]:
from qtm.gw.sigma import Sigma

sigma = Sigma.from_data(
    wfndata=wfndata,
    wfnqdata=wfnqdata,
    sigmainp=sigmainp,
    epsinp=epsinp,
    l_epsmats=epsilon.l_epsinv,
    rho=rho,
    vxc=vxc,
)

Vcoul calculation for qpts: 100%|██████████| 64/64 [00:00<00:00, 22184.75it/s]


vcoul: Vcoul:
        * gspace = <qtm.gspace.gspc.GSpace object at 0x7614b43e6850>
        * qpts = <qtm.gw.core.QPoints object at 0x7614b250ef50>
        * bare_coulomb_cutoff = 10.0
        * avgcut = 1e-05
        * l_gspace_q = <class 'list'> of length 64
        * vcoul = <class 'list'> of length 64
        * N_SAMPLES = 2500000.0
        * N_SAMPLES_COARSE = 250000.0
        * SEED = 5000
        


Vcoul calculation for qpts: 100%|██████████| 64/64 [00:39<00:00,  1.60it/s]


In [8]:
# Calculate the quasiparticle energies.
sigma.print_condition = False
results_dict = sigma.calculate_static_cohsex()

k_indices = sigma.l_k_indices
num_bands = len(results_dict[k_indices[0]]["Eqp1"])

eqp1_array = np.zeros((len(k_indices), num_bands))
for i, k_idx in enumerate(k_indices):
    eqp1_array[i, :] = results_dict[k_idx]["Eqp1"]

Sigma_CH_Static_Exact: 100%|██████████| 64/64 [00:45<00:00,  1.41it/s]


In [9]:
print("Quasiparticle energies (Eqp1):")
print_2d_matrix(eqp1_array, decimals=6)

Quasiparticle energies (Eqp1):
-8.253496  5.139970  5.139953  5.140715  8.897831  8.897884  8.894425  9.964640
-7.339045  0.890480  4.350179  4.349862  8.297971  9.884884  9.880032 13.555706
-5.577305 -2.587290  3.843615  3.843729  7.760184  9.693980  9.692948 14.035552
-7.336112  0.877843  4.347464  4.348507  8.297007  9.884039  9.885801 13.565308
-7.339655  0.891352  4.351754  4.348803  8.298169  9.886823  9.878041 13.555024
-7.018359  1.372039  3.119344  3.118319  7.219644  9.381878 12.307960 12.306783
-5.147137 -1.890927  1.325618  2.748104  7.628114 10.738291 12.306323 12.476555
-5.913765 -0.870705  1.073055  3.721370  8.704722 11.324537 11.439207 12.737638
-5.577348 -2.587346  3.843618  3.843960  7.760241  9.694585  9.692493 14.035588
-5.147387 -1.890425  1.325508  2.747967  7.627949 10.737311 12.307097 12.477251
-3.518410 -3.519542  2.013393  2.013335  6.723393  6.723884 17.162698 17.163249
-5.135391 -1.904446  1.316167  2.745173  7.627500 10.738145 12.321627 12.486144
-7.335533

In [10]:
from kernel import KernelMtxEl

q0 = [0.001, 0.001, 0.001]
l_qpts = np.array(epsinp.qpts)
l_qpts[0] *= 0
qpts = QPoints.from_cryst(wfndata.kpts.recilat, None, *l_qpts)

kernelclass = KernelMtxEl.from_BGW(
    wfndata=wfndata,
    epsinp=epsinp,
    sigmainp=sigmainp,
    q0=q0,
    l_epsmats=epsilon.l_epsinv,
    parallel=True,
)

In [11]:
data = kernelclass.kernel_mtxel()
exc = data["exc"]
head = data["head"]
body = data["body"]
wings = data["wings"]

In [12]:
numq = kernelclass.qpts.numq
numk = kernelclass.kpts.numk

In [13]:
from intkernel import InterpMtxEl

InterpClass = InterpMtxEl.from_BGW(
    wfn_finedata=wfndata,
    wfn_coarsedata=wfndata,
    epsinp=epsinp,
    sigmainp=sigmainp,
    kernel=kernelclass,
)

In [14]:
energyval = InterpClass.interp_energy(eqp1_array, 'val')
energycon = InterpClass.interp_energy(eqp1_array, 'con')

print("Interpolated quasiparticle energies (valence):")
print_2d_matrix(energyval, decimals=6)

print("Interpolated quasiparticle energies (conduction):")
print_2d_matrix(energycon, decimals=6)


Interpolated quasiparticle energies (valence):
 5.140715  5.139953  5.139970 -8.253496
 4.349862  4.350179  0.890480 -7.339045
 3.843729  3.843615 -2.587290 -5.577305
 4.348507  4.347464  0.877843 -7.336112
 4.348803  4.351754  0.891352 -7.339655
 3.118319  3.119344  1.372039 -7.018359
 2.748104  1.325618 -1.890927 -5.147137
 3.721370  1.073055 -0.870705 -5.913765
 3.843960  3.843618 -2.587346 -5.577348
 2.747967  1.325508 -1.890425 -5.147387
 2.013335  2.013393 -3.519542 -3.518410
 2.745173  1.316167 -1.904446 -5.135391
 4.350284  4.345564  0.876742 -7.335533
 3.721076  1.072537 -0.871710 -5.912721
 2.744981  1.315973 -1.903927 -5.135578
 3.111558  3.111687  1.349705 -7.011401
 4.349491  4.349223  0.887046 -7.337694
 3.118140  3.116948  1.368526 -7.016330
 2.746842  1.323477 -1.891695 -5.145104
 3.720414  1.071898 -0.872588 -5.912015
 3.117989  3.117408  1.369462 -7.016919
 4.352578  4.353646  0.898786 -7.339716
 3.723644  1.077763 -0.859091 -5.918235
 2.748613  1.326298 -1.896697 -5.

In [15]:
is_equal_val = np.allclose(energyval, np.flip(eqp1_array[:, :4], axis = -1))
is_equal_con = np.allclose(energycon, eqp1_array[:, 4:])

print(f"Interpolated quasiparticle energies (valence) are equal to the original: {is_equal_val}")
print(f"Interpolated quasiparticle energies (conduction) are equal to the original: {is_equal_con}")

Interpolated quasiparticle energies (valence) are equal to the original: True
Interpolated quasiparticle energies (conduction) are equal to the original: True


In [16]:
fine_kernel = InterpClass.interp_kernel(head, wings, body, exc, sigma.vcoul)
fine_kernel_selected = fine_kernel[:, :, 0, 0, 0, 0]
print(f"\n Selected part of fine_kernel is")

print_2d_matrix(fine_kernel_selected, decimals=6)


 Selected part of fine_kernel is
0.098670+0.000000j 0.008606-0.014370j 0.002873-0.011702j -0.005827-0.016635j -0.011140-0.007584j 0.008820+0.001504j 0.005082-0.000769j -0.006573+0.004101j 0.005294+0.008615j 0.003665-0.003196j 0.000132-0.000119j -0.004981+0.000464j 0.006268+0.012588j -0.000110-0.007741j -0.000796+0.005207j 0.009633+0.000571j 0.004081-0.001456j 0.002730-0.005802j -0.000007+0.000342j 0.000069-0.000304j 0.001438-0.006267j -0.009364-0.005171j -0.000662+0.006197j -0.003195-0.002082j -0.000400-0.000086j 0.000750+0.005607j 0.000897-0.000465j 0.003806+0.002538j 0.000414+0.000865j -0.001502-0.003039j -0.006811+0.000152j -0.001148-0.000245j -0.002195-0.001679j 0.000116-0.000093j 0.000119-0.000081j 0.000012+0.000150j 0.000274-0.000661j 0.001653-0.000272j 0.001386+0.003404j -0.006162-0.003416j -0.000956-0.000224j -0.001299-0.003550j -0.006826-0.004985j 0.001657-0.003408j 0.000676-0.000264j 0.006224-0.003121j -0.003679-0.000043j -0.001672+0.000269j -0.000986-0.004517j 0.000108-0.00

In [26]:
HBSE = InterpClass.construct_HBSE(-fine_kernel, energyval, energycon)
# print(f"\n HBSE is")
# print_2d_matrix(HBSE, decimals=6)

HGW = InterpClass.construct_HBSE(0, energyval, energycon)

In [27]:
from diag import diag_elpa

eigval_elpa, eigvec_elpa = diag_elpa(HBSE)
eigval_numpy, eigvec_numpy = np.linalg.eigh(HBSE)

print(f"\n First ten eigenvalues from ELPA are")
print(eigval_elpa[:20])
print(f"\n First ten eigenvalues from numpy are")
print(eigval_numpy[:20])




 First ten eigenvalues from ELPA are
[3.57646719 3.57999788 3.582351   3.58993019 3.59355151 3.59722773
 3.64442704 3.64665582 3.67053598 3.7065262  3.71127504 3.71447001
 3.75773502 3.76309114 3.76841221 3.77497996 3.77713301 3.83900785
 3.83950951 3.84307087]

 First ten eigenvalues from numpy are
[3.5691271  3.56974423 3.58188965 3.59339995 3.59382111 3.59503971
 3.64362152 3.6440631  3.66965723 3.69588459 3.69690311 3.70052506
 3.74811048 3.74812077 3.75114314 3.75701299 3.75715915 3.84747224
 3.84795078 3.85039438]


In [28]:
eigval_GW_elpa, eigvec_GW_elpa = diag_elpa(HGW)
eigval_GW_numpy, eigvec_GW_numpy = np.linalg.eigh(HGW)

print(f"\n First ten GW eigenvalues from ELPA are")
print(eigval_GW_elpa[:20])

print(f"\n First ten GW eigenvalues from numpy are")
print(eigval_GW_numpy[:20])



 First ten GW eigenvalues from ELPA are
[3.75370998 3.75445501 3.75447181 3.75711652 3.75716873 3.75786155
 3.75787835 3.75791376 3.75793056 3.91472611 3.91483132 3.91628089
 3.91645457 3.91653943 3.91656846 3.91659816 3.91662265 3.94331391
 3.94438153 3.94641541]

 First ten GW eigenvalues from numpy are
[3.75370998 3.75445501 3.75447181 3.75711652 3.75716873 3.75786155
 3.75787835 3.75791376 3.75793056 3.91472611 3.91483132 3.91628089
 3.91645457 3.91653943 3.91656846 3.91659816 3.91662265 3.94331391
 3.94438153 3.94641541]


In [29]:
data_eig = np.loadtxt(dirname + "eigenvalues.dat", comments="#")
eigenvalues = data_eig[:, 0]
sorted_eigenvalues = np.sort(eigenvalues)


# data_eig_noeh = np.loadtxt(dirname + "eigenvalues_noeh.dat", comments="#")
# eigenvalues_noeh = data_eig_noeh[:, 6]
# sorted_eigenvalues_noeh = np.sort(eigenvalues_noeh)

In [30]:
print(f"\n First ten eigenvalues from BGW are")
print(sorted_eigenvalues[:20])


 First ten eigenvalues from BGW are
[3.5827217 3.5829047 3.5833829 3.5950666 3.5953428 3.5964289 3.649146
 3.6500073 3.6726433 3.7133303 3.7133921 3.7136992 3.7680177 3.768151
 3.7687932 3.7729145 3.7730672 3.8436619 3.8437976 3.8443267]
